# Automated Stock Trade

In [13]:
import time
import pandas as pd
from datetime import datetime

import config
import secret
import kabucom
import data_processor

prod_mode = False

order_password = secret.ORDER_PASSWORD

In [14]:
"""
    side : 買 or 売
    is_new : 新規注文 or 返済注文
    front_order_type : 成 or 指
    成の場合、priceの指定は不要
"""
def post_order(securities_code, side, is_new, qty, front_order_type, price=0):
    if is_new == True:
        cash_margin = 2
    else:
        cash_margin = 3

    return kabucom.post_order(
        securities_code=securities_code,
        password=secret.ORDER_PASSWORD,
        side=data_processor.side_dict[side],
        cash_margin=cash_margin, 
        margin_trade_type=3, # デイトレ
        qty=qty,
        front_order_type=data_processor.front_order_type_dict[front_order_type],
        price=price,
        prod_mode=prod_mode
    )

In [15]:
def get_board_dataframe(sc, df):
    try:
        res = kabucom.get_board(sc)
        tmp_df = data_processor.convert_to_df(res)
        df = pd.concat([df, tmp_df])
        return df
    except Exception as e:                
        print("Failed to gather board: " + str(sc) + " " + str(datetime.now()) + str(e))
        return df

In [16]:
use_col = 'SMA_Cross_5_14_diff'

"""
    top-sell
"""
def strategy_top_sell(series, threshold):
    if series[use_col] >= threshold:
        return 'sell'
    elif series[use_col] <= 1-threshold:
        return 'buy'
    else:
        return 'watch'

"""
    top-buy
"""
def strategy_top_buy(series, threshold):
    if series[use_col] >= threshold:
        return 'buy'
    elif series[use_col] <= 1-threshold:
        return 'sell'
    else:
        return 'watch'

In [17]:
def create_features(df):
    def min_max(x, axis=None):
        min = x.min(axis=axis, keepdims=True)
        max = x.max(axis=axis, keepdims=True)
        result = (x-min)/(max-min)
        return result
        
    def apply_min_max(df, col):
        return pd.DataFrame(min_max((df[col]).values), columns=[col], index=df.index)[col]
    
    # FIXME: 特定scで絞ると何故か行数が減る
    specific_df = df.sort_values(by=['Sell1Time'], ascending=True).set_index('Sell1Time')

    specific_df['CalcPrice5mean'] = specific_df[['CalcPrice']].rolling(window=5*60, min_periods=10).mean()
    specific_df['CalcPrice14mean'] = specific_df[['CalcPrice']].rolling(window=14*60, min_periods=10).mean()

    # プラス値 .. 短期が中期より上
    # マイナス値 .. 中期が短期より上
    specific_df['SMA_Cross_5_14'] = (specific_df['CalcPrice5mean'] - specific_df['CalcPrice14mean'])
    specific_df['SMA_Cross_5_14_diff'] = specific_df['SMA_Cross_5_14'].diff()
    specific_df['SMA_Cross_5_14_diff'] = apply_min_max(specific_df.fillna(0), 'SMA_Cross_5_14_diff')

    return specific_df

In [18]:
def take_action(action, positions, status, sc, qty):
    if action == 'buy':
        # positionsに銘柄の情報がない時、新規で購入する
        if sc not in positions:
            try:
                order = post_order(sc, 'buy', is_new=True, qty=qty, front_order_type='market')
                positions[sc] = {
                    'SecuritiesCode': sc,
                    'Price': order['Price'],
                    'BuySell': 'buy'
                }
                status = 'bought'
            except:
                status = None
            return positions, status

        # positionsに銘柄の情報がある時、売りから入ってたら返済する
        else:
            if status == 'sold':
                try:
                    order = post_order(sc, 'buy', is_new=False, qty=qty, front_order_type='market')
                    positions.pop(sc)
                    status = None
                except:
                    status = 'sold'
                return positions, status

    elif action == 'sell':
        # positionsに銘柄の情報がない時、新規で空売りする
        if sc not in positions:
            try:
                order = post_order(sc, 'sell', is_new=True, qty=qty, front_order_type='market')
                positions[sc] = {
                    'SecuritiesCode': sc,
                    'Price': order['Price'],
                    'BuySell': 'sell'
                }
                status = 'sold'
            except:
                status = None
            return positions, status

        # positionsに銘柄の情報がある時、買いから入ってたら返済する
        else:
            if status == 'bought':
                try:
                    order = post_order(sc, 'sell', is_new=False, qty=qty, front_order_type='market')
                    positions.pop(sc)
                    status = None
                except:
                    status = 'bought'
                return positions, status            
    # watch
    else:
        return positions, status

In [19]:
def get_positions_and_status(sc, qty):
    buy_positions = kabucom.get_positions(
        product=2,
        securities_code=sc, 
        side=data_processor.side_dict['buy'], 
        addinfo='false',
        prod_mode=prod_mode
    )
    if data_processor.has_positions(buy_positions, sc, qty):
        return buy_positions, 'bought'
    
    sell_positions = kabucom.get_positions(
        product=2,
        securities_code=sc, 
        side=data_processor.side_dict['sell'], 
        addinfo='false',
        prod_mode=prod_mode
    )
    if data_processor.has_positions(sell_positions, sc, qty):    
        return sell_positions, 'sold'

    return {}, None

In [20]:
import warnings
warnings.simplefilter('ignore')

def autotrade(sc, qty, threshold=0.8, verbose=False):
    def right_before_trade_end(dt_now):
        return datetime(dt_now.year, dt_now.month, dt_now.day, 15, 29, 0) <= dt_now and dt_now < datetime(dt_now.year, dt_now.month, dt_now.day, 15, 00, 0)
    def before_trade(dt_now):
        return dt_now < datetime(dt_now.year, dt_now.month, dt_now.day, 9, 0, 0)
    def during_lunch_time(dt_now):
        return datetime(dt_now.year, dt_now.month, dt_now.day, 11, 30, 0) <= dt_now and dt_now < datetime(dt_now.year, dt_now.month, dt_now.day, 12, 30, 0)
    def during_trade(dt_now):
        return datetime(dt_now.year, dt_now.month, dt_now.day, 9, 0, 0) <= dt_now and dt_now < datetime(dt_now.year, dt_now.month, dt_now.day, 15, 0, 0)

    """
        {
            "SecuritiesCode": "9104",
            "Price": 3305.5,
            "BuySell": "buy"
        }
    """
    action_history = []
    status_history = []
    df = data_processor.get_data_or_empty_dataframe()
    
    while True:
        # 時刻チェック
        time.sleep(1) 
        dt_now = datetime.now()
        if right_before_trade_end(dt_now):
            # ポジションを無くして返済する
            print("返済する")
            continue
        elif before_trade(dt_now) or during_lunch_time(dt_now):
            continue
        elif not during_trade(dt_now):
            break

        # 板情報を取得
        df = get_board_dataframe(sc, df)

        # 取引特徴量の作成
        df = create_features(df, sc)

        data_processor.save(df)
        
        # 状況を見て、行動を決定
        action = strategy_top_sell(df.iloc[0], threshold)
        if verbose:
            print(action, df.iloc[0].SMA_Cross_5_14_diff, df.iloc[0].name)
        action_history.append({
            "Time": dt_now,
            "Action": action,
            "SMA_Cross_5_14_diff": df.iloc[0].SMA_Cross_5_14_diff
        })
        if action == 'watch':
            continue

        # 行動する場合、保有銘柄と状態を取得
        positions, status = get_positions_and_status(sc, qty)
        if verbose:
            print(dt_now, positions, status)
        status_history.append({
            "SecuritiesCode": sc,
            "Qty":qty,
            "Status": status
        })
        # 行動
        if prod_mode:
            positions, status = take_action(action, positions, status, sc, qty)
      
    return df, action_history, status_history

In [21]:
sc = '7203'
qty = 100

df, action_history, status_history = autotrade(sc, qty)

In [22]:
df

,LowPrice,UnderBuyQty,SymbolName,CurrentPriceTime,BidPrice,ChangePreviousClose,ChangePreviousClosePer,Sell6Price,Sell6Qty,Buy4Price,...,Buy3Qty,CurrentPrice,Buy10Price,Buy10Qty,TotalMarketValue,HighPrice,CalcPrice5mean,CalcPrice14mean,SMA_Cross_5_14,SMA_Cross_5_14_diff
0,2007.5,6295000.0,トヨタ自動車,2022-05-20T14:59:58+09:00,2050.0,45.0,2.24,2052.5,55000.0,2047.0,...,15200.0,2050.0,2044.0,80600.0,3.344572e+13,2052.5,NaN,NaN,NaN,0.477509
1,2007.5,6375100.0,トヨタ自動車,2022-05-20T14:59:57+09:00,2050.5,44.0,2.19,2053.0,57900.0,2047.5,...,6400.0,2049.0,2044.5,7600.0,3.342941e+13,2052.5,NaN,NaN,NaN,0.477509
2,2007.5,6383200.0,トヨタ自動車,2022-05-20T14:59:57+09:00,2050.5,45.0,2.24,2053.0,58900.0,2048.0,...,2600.0,2050.0,2045.0,64800.0,3.344572e+13,2052.5,NaN,NaN,NaN,0.477509
3,2007.5,6383400.0,トヨタ自動車,2022-05-20T14:59:55+09:00,2050.5,44.5,2.22,2053.0,62200.0,2048.0,...,6100.0,2049.5,2045.0,64800.0,3.345388e+13,2052.5,NaN,NaN,NaN,0.477509
4,2007.5,6383400.0,トヨタ自動車,2022-05-20T14:59:53+09:00,2050.5,45.0,2.24,2053.0,62200.0,2048.0,...,6100.0,2050.0,2045.0,64800.0,3.344572e+13,2052.5,NaN,NaN,NaN,0.477509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,2007.5,5716900.0,トヨタ自動車,2022-05-20T13:44:36+09:00,2037.5,32.5,1.62,2040.0,42300.0,2035.5,...,14600.0,2037.5,2032.5,12600.0,3.324179e+13,2040.5,2038.433333,2037.760119,0.673214,0.498270
2732,2007.5,5716900.0,トヨタ自動車,2022-05-20T13:44:36+09:00,2037.5,32.5,1.62,2040.0,42300.0,2035.5,...,14600.0,2037.5,2032.5,12600.0,3.324179e+13,2040.5,2038.431667,2037.757738,0.673929,0.498270
2733,2007.5,5716900.0,トヨタ自動車,2022-05-20T13:44:36+09:00,2037.5,32.5,1.62,2040.0,42300.0,2035.5,...,14600.0,2037.5,2032.5,12600.0,3.324179e+13,2040.5,2038.430000,2037.755357,0.674643,0.498270
2734,2007.5,5716900.0,トヨタ自動車,2022-05-20T13:44:36+09:00,2037.5,32.5,1.62,2040.0,42300.0,2035.5,...,14600.0,2037.5,2032.5,12600.0,3.324179e+13,2040.5,2038.428333,2037.752976,0.675357,0.498270


In [23]:
action_history

[]

In [24]:
status_history

[]